# TP : Compression d'images avec JPEG 2000 - CORRIGÉ

**Niveau :** Première NSI

**Objectifs :**
- Comprendre le principe de la compression d'images
- Manipuler des tableaux représentant des pixels
- Implémenter la transformation de Haar
- Analyser une image à différentes échelles

## Introduction : Pourquoi compresser une image ?

### Le problème

Une image de **816 × 608 pixels** en couleur nécessite :
- 3 octets par pixel (rouge, vert, bleu)
- Soit : 816 × 608 × 3 = **1 488 384 octets** ≈ **1,49 Mo**

Avec la compression JPEG, cette même photo ne pèse que **237 Ko**, soit environ **6,28 fois moins** !

### L'astuce

Dans une région uniforme (ciel bleu, mur blanc), il est plus efficace de stocker :
- La couleur globale
- Seulement les pixels qui diffèrent

Le format **JPEG 2000** utilise une technique mathématique appelée **transformation en ondelettes** pour analyser et compresser les images.

## Importation des bibliothèques

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Partie 1 : Transformation 1D - Moyennes et différences

### Principe de base

Au lieu de stocker deux nombres `a` et `b`, on peut stocker :
- Leur **moyenne** : $x = \frac{a+b}{2}$
- La **moitié de leur différence** : $y = \frac{b-a}{2}$

On peut retrouver `a` et `b` avec :
- $b = x + y$
- $a = x - y$

**Intérêt** : Si `a` et `b` sont proches, alors `y` sera très petit et pourra être arrondi à 0 !

### 📝 Exercice 1 : Transformation de deux pixels

Voici les tons de gris de deux pixels voisins : `a = 25` et `b = 23`.

In [ ]:
# Deux pixels voisins
a = 25
b = 23

# SOLUTION: Calculer la moyenne x
x = (a + b) / 2

# SOLUTION: Calculer la demi-différence y
y = (b - a) / 2

print(f"Pixels originaux : a={a}, b={b}")
print(f"Transformés : moyenne={x}, demi-différence={y}")

### 📝 Exercice 2 : Vérification de la reconstruction

In [ ]:
# SOLUTION: Reconstruire b et a à partir de x et y
b_reconstruit = x + y
a_reconstruit = x - y

print(f"Reconstruction : a={a_reconstruit}, b={b_reconstruit}")
print(f"Vérification : a original={a}, b original={b}")

### 📝 Exercice 3 : Transformation d'une ligne de pixels

Appliquons cette transformation sur 16 pixels (positions 212 à 227 de la ligne 220 de la photo).

In [ ]:
# Ligne de 16 pixels
pixels = [25, 23, 23, 22, 31, 115, 124, 125, 130, 127, 138, 222, 222, 228, 229, 229]

# Initialisation des listes pour moyennes et demi-différences
moyennes = []
demi_differences = []

# SOLUTION: Parcourir les pixels 2 par 2 et calculer moyennes et demi-différences
for i in range(0, len(pixels), 2):
    a = pixels[i]
    b = pixels[i+1]
    
    # Calcul de la moyenne
    x = (a + b) / 2
    moyennes.append(x)
    
    # Calcul de la demi-différence
    y = (b - a) / 2
    demi_differences.append(y)

print("Pixels originaux :", pixels)
print("\nMoyennes :", moyennes)
print("Demi-différences :", demi_differences)

### 🔍 Observation

Que remarquez-vous ?
- Plusieurs demi-différences sont **très petites** (-1, -0.5, 0.5, -1.5, 0)
- Les grandes différences (42, 42, 3) indiquent des **changements brusques** de couleur
- Si on arrondit les petites valeurs à 0, on **compresse** l'information !

## Partie 2 : Transformation 2D sur un carré 2×2

### Le principe en 2 dimensions

Pour un carré 2×2 de pixels :

```
a  b
c  d
```

On calcule 4 valeurs :

1. **Rouge (moyenne globale)** : $\frac{a+b+c+d}{4}$
2. **Bleu (différences verticales)** : $\frac{(b-a)+(d-c)}{4}$
3. **Vert (différences horizontales)** : $\frac{(c-a)+(d-b)}{4}$
4. **Violet (différences obliques)** : $\frac{(b+c)-(a+d)}{4}$

### 📝 Exercice 4 : Implémenter la transformation 2D

In [ ]:
def transformation_haar_2x2(carre):
    """
    Applique la transformation de Haar sur un carré 2x2.
    
    Paramètres:
        carre: tableau numpy 2x2
    
    Retourne:
        tableau 2x2 transformé avec:
        [rouge, bleu]
        [vert, violet]
    """
    a, b = carre[0, 0], carre[0, 1]
    c, d = carre[1, 0], carre[1, 1]
    
    # SOLUTION: Calculer les 4 coefficients
    rouge = (a + b + c + d) / 4  # Moyenne globale
    bleu = ((b - a) + (d - c)) / 4  # Différences verticales
    vert = ((c - a) + (d - b)) / 4  # Différences horizontales
    violet = ((b + c) - (a + d)) / 4  # Différences obliques
    
    return np.array([[rouge, bleu], [vert, violet]])

### Test de la fonction

In [ ]:
# Exemple du document : un petit carré 2x2 extrait de la photo
carre_test = np.array([[134, 224],
                       [137, 162]], dtype=float)

resultat = transformation_haar_2x2(carre_test)

print("Carré original :")
print(carre_test)
print("\nCarré transformé :")
print(resultat)
print("\nDétail :")
print(f"Rouge (moyenne) : {resultat[0,0]:.2f}")
print(f"Bleu (vertical) : {resultat[0,1]:.2f}")
print(f"Vert (horizontal) : {resultat[1,0]:.2f}")
print(f"Violet (oblique) : {resultat[1,1]:.2f}")

## Partie 3 : Application sur une image 8×8

Nous allons maintenant appliquer la transformation sur une petite image de 8×8 pixels extraite de la photo.

In [ ]:
# Image 8x8 (pixels 226-233 horizontalement et 216-223 verticalement)
image_8x8 = np.array([
    [126, 130, 126, 134, 221, 231, 228, 230],
    [120, 127, 120, 181, 234, 229, 235, 234],
    [123, 127, 134, 224, 231, 228, 207, 179],
    [126, 116, 137, 162, 135, 112, 64, 22],
    [119, 135, 114, 17, 24, 48, 78, 96],
    [108, 202, 182, 86, 127, 125, 125, 122],
    [146, 234, 194, 108, 128, 124, 131, 134],
    [185, 234, 205, 115, 130, 123, 124, 120]
], dtype=float)

# Visualisation de l'image originale
plt.figure(figsize=(6, 6))
plt.imshow(image_8x8, cmap='gray', vmin=0, vmax=255)
plt.title("Image originale 8×8")
plt.colorbar(label="Niveau de gris")
plt.show()

### 📝 Exercice 5 : Transformation complète d'une image 8×8

In [ ]:
def transformation_haar_image(image):
    """
    Applique la transformation de Haar sur une image de taille 2^n × 2^m.
    Divise l'image en blocs 2×2 et applique la transformation.
    
    Retourne 4 images de taille n×m :
    - rouge (moyennes)
    - bleu (différences verticales)
    - vert (différences horizontales)
    - violet (différences obliques)
    """
    hauteur, largeur = image.shape
    nouvelle_hauteur = hauteur // 2
    nouvelle_largeur = largeur // 2
    
    # Initialisation des 4 quadrants
    rouge = np.zeros((nouvelle_hauteur, nouvelle_largeur))
    bleu = np.zeros((nouvelle_hauteur, nouvelle_largeur))
    vert = np.zeros((nouvelle_hauteur, nouvelle_largeur))
    violet = np.zeros((nouvelle_hauteur, nouvelle_largeur))
    
    # SOLUTION: Parcourir l'image par blocs 2×2
    for i in range(0, hauteur, 2):
        for j in range(0, largeur, 2):
            # Extraire le bloc 2×2
            bloc = image[i:i+2, j:j+2]
            
            # Appliquer la transformation
            transforme = transformation_haar_2x2(bloc)
            
            # Stocker dans les quadrants
            rouge[i//2, j//2] = transforme[0, 0]
            bleu[i//2, j//2] = transforme[0, 1]
            vert[i//2, j//2] = transforme[1, 0]
            violet[i//2, j//2] = transforme[1, 1]
    
    # Reconstruction de l'image transformée
    resultat = np.zeros_like(image)
    resultat[0:nouvelle_hauteur, 0:nouvelle_largeur] = rouge
    resultat[0:nouvelle_hauteur, nouvelle_largeur:largeur] = bleu
    resultat[nouvelle_hauteur:hauteur, 0:nouvelle_largeur] = vert
    resultat[nouvelle_hauteur:hauteur, nouvelle_largeur:largeur] = violet
    
    return resultat, rouge, bleu, vert, violet

In [ ]:
# Application de la transformation
transformee, rouge, bleu, vert, violet = transformation_haar_image(image_8x8)

# Visualisation des résultats
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Image originale
axes[0, 0].imshow(image_8x8, cmap='gray', vmin=0, vmax=255)
axes[0, 0].set_title("Image originale")
axes[0, 0].axis('off')

# Carré rouge (moyennes)
axes[0, 1].imshow(rouge, cmap='gray', vmin=0, vmax=255)
axes[0, 1].set_title("Rouge : Moyennes (structure)")
axes[0, 1].axis('off')

# Transformation complète
axes[0, 2].imshow(transformee, cmap='gray')
axes[0, 2].set_title("Image transformée complète")
axes[0, 2].add_patch(plt.Rectangle((0, 0), 4, 4, fill=False, edgecolor='red', linewidth=2))
axes[0, 2].add_patch(plt.Rectangle((4, 0), 4, 4, fill=False, edgecolor='blue', linewidth=2))
axes[0, 2].add_patch(plt.Rectangle((0, 4), 4, 4, fill=False, edgecolor='green', linewidth=2))
axes[0, 2].add_patch(plt.Rectangle((4, 4), 4, 4, fill=False, edgecolor='purple', linewidth=2))
axes[0, 2].axis('off')

# Pour visualiser les différences, on ajoute 127.5 pour centrer
axes[1, 0].imshow(bleu + 127.5, cmap='gray', vmin=0, vmax=255)
axes[1, 0].set_title("Bleu : Différences verticales")
axes[1, 0].axis('off')

axes[1, 1].imshow(vert + 127.5, cmap='gray', vmin=0, vmax=255)
axes[1, 1].set_title("Vert : Différences horizontales")
axes[1, 1].axis('off')

axes[1, 2].imshow(violet + 127.5, cmap='gray', vmin=0, vmax=255)
axes[1, 2].set_title("Violet : Différences obliques")
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("Carré rouge (moyennes 4×4) :")
print(np.round(rouge))

### 🔍 Analyse

- Le **carré rouge** contient la **structure générale** de l'image
- Le **carré bleu** met en évidence les **contours verticaux**
- Le **carré vert** met en évidence les **contours horizontaux**
- Le **carré violet** met en évidence les **contours obliques**

## Partie 4 : Itérations multiples (Pour aller plus loin)

On peut **réappliquer** la transformation sur le carré rouge pour analyser l'image à différentes échelles !

In [ ]:
def transformation_haar_multi_niveaux(image, niveaux=3):
    """
    Applique la transformation de Haar plusieurs fois.
    À chaque itération, on transforme le quadrant rouge (en haut à gauche).
    """
    resultat = image.copy()
    hauteur, largeur = image.shape
    
    for niveau in range(niveaux):
        # Taille du quadrant à transformer
        h = hauteur // (2 ** niveau)
        l = largeur // (2 ** niveau)
        
        # Extraire le quadrant rouge actuel
        quadrant = resultat[0:h, 0:l]
        
        # Appliquer la transformation
        transforme, _, _, _, _ = transformation_haar_image(quadrant)
        
        # Remplacer dans l'image résultat
        resultat[0:h, 0:l] = transforme
    
    return resultat

In [ ]:
# Application de 3 niveaux de transformation
image_multi = transformation_haar_multi_niveaux(image_8x8, niveaux=3)

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(image_8x8, cmap='gray', vmin=0, vmax=255)
axes[0].set_title("Image originale 8×8")
axes[0].axis('off')

axes[1].imshow(image_multi, cmap='gray')
axes[1].set_title("Transformation multi-niveaux (3 itérations)")
axes[1].axis('off')

plt.tight_layout()
plt.show()

## Partie 5 : Compression avec perte d'information

Pour compresser vraiment, on peut **arrondir à zéro** les petites valeurs !

In [ ]:
def compresser(image_transformee, seuil=10):
    """
    Compresse l'image en mettant à zéro toutes les valeurs dont
    la valeur absolue est inférieure au seuil.
    """
    compresse = image_transformee.copy()
    
    # SOLUTION: Mettre à zéro les petites valeurs
    # On utilise un masque booléen pour identifier les valeurs à mettre à zéro
    masque = np.abs(compresse) < seuil
    compresse[masque] = 0
    
    return compresse

In [ ]:
# Test avec différents seuils
seuils = [5, 15, 30]

fig, axes = plt.subplots(1, len(seuils)+1, figsize=(16, 4))

axes[0].imshow(image_multi, cmap='gray')
axes[0].set_title("Transformée (sans compression)")
axes[0].axis('off')

for i, seuil in enumerate(seuils):
    compresse = compresser(image_multi, seuil)
    nb_zeros = np.sum(compresse == 0)
    pourcentage = (nb_zeros / compresse.size) * 100
    
    axes[i+1].imshow(compresse, cmap='gray')
    axes[i+1].set_title(f"Seuil = {seuil}\n{pourcentage:.1f}% de zéros")
    axes[i+1].axis('off')

plt.tight_layout()
plt.show()

## 🎯 Questions de réflexion

1. **Pourquoi la transformation en moyennes et différences est-elle utile pour la compression ?**

   *Réponse :* Dans les zones uniformes d'une image, les pixels voisins ont des valeurs très proches. La transformation produit alors de grandes moyennes (qui représentent la structure) et de petites différences (qui peuvent être arrondies à 0 pour compresser).

2. **Quel est le rôle de chacun des 4 quadrants (rouge, bleu, vert, violet) ?**

   *Réponse :*
   - Rouge : contient la structure générale (moyennes)
   - Bleu : détecte les changements verticaux
   - Vert : détecte les changements horizontaux
   - Violet : détecte les changements en diagonale

3. **Que se passe-t-il si on augmente le seuil de compression ?**

   *Réponse :* Plus on augmente le seuil, plus on met de valeurs à zéro, donc meilleure est la compression. Mais on perd aussi plus de détails de l'image originale.

4. **Quels types d'images se compressent le mieux avec cette méthode ?**

   *Réponse :* Les images avec de grandes zones uniformes (ciel, murs, aplats de couleur) se compressent mieux car elles produisent beaucoup de petites différences.

5. **Quelle est la différence entre compression avec et sans perte ?**

   *Réponse :* 
   - Sans perte : on peut reconstruire exactement l'image originale
   - Avec perte : on arrondit des valeurs, donc l'image reconstruite est légèrement différente (mais souvent imperceptible à l'œil)

## 🚀 Pour aller plus loin

### Exercice bonus 1 : Reconstruction de l'image

In [ ]:
def reconstruction_haar_2x2(carre_transforme):
    """
    Reconstruction inverse de la transformation de Haar.
    
    À partir de [rouge, bleu; vert, violet],
    retrouver [a, b; c, d]
    """
    # SOLUTION:
    # On a les équations:
    # rouge = (a+b+c+d)/4
    # bleu = ((b-a)+(d-c))/4 = (b+d-a-c)/4
    # vert = ((c-a)+(d-b))/4 = (c+d-a-b)/4
    # violet = ((b+c)-(a+d))/4 = (b+c-a-d)/4
    #
    # En résolvant ce système:
    # a = rouge - bleu - vert - violet
    # b = rouge + bleu - vert + violet
    # c = rouge - bleu + vert + violet
    # d = rouge + bleu + vert - violet
    
    rouge = carre_transforme[0, 0]
    bleu = carre_transforme[0, 1]
    vert = carre_transforme[1, 0]
    violet = carre_transforme[1, 1]
    
    a = rouge - bleu - vert - violet
    b = rouge + bleu - vert + violet
    c = rouge - bleu + vert + violet
    d = rouge + bleu + vert - violet
    
    return np.array([[a, b], [c, d]])

# Test de la reconstruction
carre_original = np.array([[134, 224], [137, 162]], dtype=float)
carre_trans = transformation_haar_2x2(carre_original)
carre_reconstruit = reconstruction_haar_2x2(carre_trans)

print("Carré original :")
print(carre_original)
print("\nCarré transformé :")
print(carre_trans)
print("\nCarré reconstruit :")
print(carre_reconstruit)
print("\nDifférence (doit être proche de 0) :")
print(np.max(np.abs(carre_original - carre_reconstruit)))

In [ ]:
def reconstruction_haar_image(image_transformee):
    """
    Reconstruction complète d'une image transformée.
    """
    hauteur, largeur = image_transformee.shape
    nouvelle_hauteur = hauteur // 2
    nouvelle_largeur = largeur // 2
    
    # Extraire les 4 quadrants
    rouge = image_transformee[0:nouvelle_hauteur, 0:nouvelle_largeur]
    bleu = image_transformee[0:nouvelle_hauteur, nouvelle_largeur:largeur]
    vert = image_transformee[nouvelle_hauteur:hauteur, 0:nouvelle_largeur]
    violet = image_transformee[nouvelle_hauteur:hauteur, nouvelle_largeur:largeur]
    
    # Reconstruire l'image
    image_reconstruite = np.zeros_like(image_transformee)
    
    for i in range(nouvelle_hauteur):
        for j in range(nouvelle_largeur):
            carre_trans = np.array([
                [rouge[i, j], bleu[i, j]],
                [vert[i, j], violet[i, j]]
            ])
            carre_reconstruit = reconstruction_haar_2x2(carre_trans)
            
            image_reconstruite[2*i:2*i+2, 2*j:2*j+2] = carre_reconstruit
    
    return image_reconstruite

# Test de reconstruction
image_reconstruite = reconstruction_haar_image(transformee)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(image_8x8, cmap='gray', vmin=0, vmax=255)
axes[0].set_title("Image originale")
axes[0].axis('off')

axes[1].imshow(transformee, cmap='gray')
axes[1].set_title("Image transformée")
axes[1].axis('off')

axes[2].imshow(image_reconstruite, cmap='gray', vmin=0, vmax=255)
axes[2].set_title("Image reconstruite")
axes[2].axis('off')

plt.tight_layout()
plt.show()

print(f"Erreur maximale de reconstruction : {np.max(np.abs(image_8x8 - image_reconstruite)):.10f}")

### Exercice bonus 2 : Charger une vraie image

In [ ]:
# SOLUTION: Créer une image simple (damier)
damier = np.zeros((64, 64))
for i in range(8):
    for j in range(8):
        if (i + j) % 2 == 0:
            damier[i*8:(i+1)*8, j*8:(j+1)*8] = 255

plt.figure(figsize=(6, 6))
plt.imshow(damier, cmap='gray', vmin=0, vmax=255)
plt.title("Damier 64×64")
plt.axis('off')
plt.show()

# Appliquer la transformation de Haar
damier_transforme = transformation_haar_multi_niveaux(damier, niveaux=6)

# Compresser avec différents seuils
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(damier, cmap='gray', vmin=0, vmax=255)
axes[0].set_title("Damier original")
axes[0].axis('off')

axes[1].imshow(damier_transforme, cmap='gray')
axes[1].set_title("Transformé (6 niveaux)")
axes[1].axis('off')

for i, seuil in enumerate([10, 50]):
    compresse = compresser(damier_transforme, seuil)
    nb_zeros = np.sum(compresse == 0)
    pourcentage = (nb_zeros / compresse.size) * 100
    
    axes[i+2].imshow(compresse, cmap='gray')
    axes[i+2].set_title(f"Compressé (seuil={seuil})\n{pourcentage:.1f}% zéros")
    axes[i+2].axis('off')

plt.tight_layout()
plt.show()

### Exercice bonus 3 : Analyse de la compression

In [ ]:
# SOLUTION: Analyser le taux de compression en fonction du seuil
seuils = range(0, 101, 5)
taux_zeros = []

for seuil in seuils:
    compresse = compresser(image_multi, seuil)
    pourcentage = (np.sum(compresse == 0) / compresse.size) * 100
    taux_zeros.append(pourcentage)

plt.figure(figsize=(10, 6))
plt.plot(seuils, taux_zeros, 'b-', linewidth=2)
plt.xlabel("Seuil de compression")
plt.ylabel("Pourcentage de coefficients mis à zéro (%)")
plt.title("Taux de compression en fonction du seuil")
plt.grid(True, alpha=0.3)
plt.show()

print("Analyse du taux de compression :")
print(f"Seuil 10 : {taux_zeros[2]:.1f}% de zéros")
print(f"Seuil 50 : {taux_zeros[10]:.1f}% de zéros")
print(f"Seuil 100 : {taux_zeros[20]:.1f}% de zéros")

## 📚 Pour en savoir plus

- Le format **JPEG 2000** utilise des ondelettes plus sophistiquées que celle de Haar
- Les **ondelettes de Daubechies** sont les plus utilisées en pratique
- La mathématicienne **Ingrid Daubechies** a reçu le prix Wolf de mathématiques en 2023
- JPEG 2000 est utilisé en imagerie médicale, météorologie, et par les professionnels de l'image

### Applications réelles de JPEG 2000

1. **Imagerie médicale** : préserve les détails importants pour les diagnostics
2. **Météorologie** : format GRIB2 pour les prévisions mondiales
3. **Cinéma numérique** : distribution de films en haute qualité
4. **Archives** : conservation d'images patrimoniales

---

**Source :** Article de Christiane Rousseau, Université de Montréal  
**Adapté pour 1ère NSI**  
**Version :** Corrigé complet